In [146]:
import numpy as np
import json
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.stats import multivariate_normal
from scipy.stats import invwishart as iw
from data_fusion import Data_fusion
import matplotlib.pyplot as plt
from dict_search import gen_dict_extract

zero = np.zeros((3,1))
df = 10

def inv(M):
    if M.size == 1:
        return M**(-1)
    else:
        return np.linalg.inv(M)
    

def normalize(v):
    return v * (np.linalg.norm(v))**(-1)

def MSE(u, v):
    return  ((u - v)**2).mean(axis=None)

In [147]:
def plot_data_inputs(robot_locs, satellite_locs):
    pass
#     fig = plt.figure()
#     ax = plt.axes(projection='3d')
#     ax.scatter(robot_locs[:,0], robot_locs[:,1], robot_locs[:,2])
#     ax.scatter(satellite_locs[:,0], satellite_locs[:,1], satellite_locs[:,2])

In [148]:
class Data():
    def __init__(self, Y, H, std):
        self.Y = Y
        self.H = H
        self.std = std
        self.batch_ind = 0
    
    def getY(self):
        return self.Y
    
    def getH(self):
        return self.H
    
    def getStdDev(self):
        return self.std
    
    def completed(self):
        return self.batch_ind == self.Y.size
    
    #returns R and not s
    def getNextBatch(self):
        self.batch_ind += 1
        return self.H[self.batch_ind - 1].reshape(1,3), self.Y[self.batch_ind - 1][0], self.std[self.batch_ind - 1][0]**2

# Data Generation

###### Reads in "node_config.json" and computes H vectors (vectors that connect emitter and observer) 

In [149]:
def get_H(robot_num):
    robot_loc = np.array(json.load(open("robot_poses.json"))["positions"])
    n = 0
    fin = open("node_config.json") 
    inp = json.load(fin)
    corr_s = [inp['emitters']['GROUND']['corr'], 
              inp['emitters']['GPS']['corr'], 
              inp['emitters']['GLONASS']['corr'], 
              inp['emitters']['GALILEO']['corr']]
    
    emitters = np.array([inp['emitters']['GROUND']['data'], 
                        inp['emitters']['GPS']['data'],
                        inp['emitters']['GLONASS']['data'],
                        inp['emitters']['GALILEO']['data']]).reshape(4*len(inp['emitters']['GROUND']['data']), 3)

    
    fusion = Data_fusion(inp['technique'])

    H = np.zeros((len(emitters), 3))
    i = 0
    for node in emitters:
        H[i,:][:3] = np.array(node) - robot_loc[robot_num]
        i+=1
        
    
    plot_data_inputs(robot_loc, np.array(emitters))
    
    return H, len(emitters), fusion, corr_s

def generate_R(corr, n):
    diag = np.random.rand(n, 1).reshape(n, 1, 1)
    R = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1):
            R[i][j] = np.sqrt(diag[i])*np.sqrt(diag[j])*corr[int(i/10)]
            R[j][i] = R[i, j]
    return R

###### Returns projected velocity along corresponding geometric vector for emitter (H--> geometric vector)

In [150]:
def get_true_Y(H):
    true_mu = np.random.multivariate_normal(zero.reshape(3,), 50*np.identity(3)).reshape(3,1)
    return (H@true_mu).reshape(H.shape[0], 1), true_mu

###### Returns "observed" y value that adds error to the true y based on the randomly generated R variance

In [155]:
def get_y(true_Y, R):
    Y = np.zeros((n , 1))
    for i in range(true_Y.size):
#         Y[i] = np.random.multivariate_normal(true_Y[i], np.array([[R[i]]]))
        Y[i] = true_Y[i]
    return Y

## Fusion Integration

In [156]:
def getInitialInformationMatrix(data):
    I = np.zeros((3, 3))
    i = np.zeros((3, 1))
    for _ in range(10):
        H, y, R = data.getNextBatch()
        I = I + H.T @ H * inv(R) 
        i = i + H.T * inv(R) * y
    return I, i

    
    
def add_satellite(data, I, i, fusion):
    H, y, R = data.getNextBatch()

    #compute state dimensional from information 
    C = np.linalg.inv(I)
    mu = C @ i
    
    #normalize H first
    u = H
    
    #project mat/vec in direction of u
    mu_a = u @ mu
    C_a = u @ C @ u.T

    #set-up b distribution
    mu_b = np.array(y).reshape(1, 1)
    C_b = np.array(R).reshape(1, 1)
    
    
    x_f, C_f = fusion.fuse(mu_a, mu_b, C_a, C_b)

    
    #Compute additional information
    D = inv(inv(C_f) - inv(C_a))
    x_d = D @ (inv(C_f) @ x_f - inv(C_a) @ mu_a)
    
    
    fused_I = I + inv(D) * u.T @ u
    fused_i = i + u.T * inv(D) * x_d
    return fused_I, fused_i

In [157]:
mse = []
anees = []
for i in range(100):
    H, n, fusion, corr = get_H(i)
    true_Y, mu = get_true_Y(H)
    R = generate_R(corr, n)
    if (i ==0):
        print(R)

    Y = get_y(true_Y, np.diag(R))

    data = Data(Y, H, R)

    info_mat, info_vec = getInitialInformationMatrix(data)
    while not data.completed():
        info_mat, info_vec = add_satellite(data, info_mat, info_vec, fusion)
    pred = ((inv(info_mat) @ info_vec))
    covs = np.sqrt(np.diag(inv(info_mat)))[:3].reshape(3,)
    error = pred - mu
    
    mse.append(MSE(mu, pred))
    anees.append(error.T @ info_mat @ error)

[[0.04304989 0.10543066 0.08773733 0.06769193 0.08291578 0.10722612
  0.0941797  0.04746212 0.06771316 0.0548048  0.04169604 0.07021688
  0.04781792 0.06364787 0.05012488 0.06364468 0.07358787 0.06984107
  0.03994683 0.07701838 0.25715858 0.24683067 0.20270688 0.25155094]
 [0.10543066 0.25820331 0.21487174 0.16577987 0.20306359 0.26260045
  0.23064932 0.11623635 0.16583184 0.13421883 0.10211503 0.17196353
  0.11710773 0.15587582 0.12275754 0.155868   0.18021921 0.17104319
  0.09783115 0.18862064 0.62979017 0.60449678 0.4964361  0.61605686]
 [0.08773733 0.21487174 0.17881205 0.13795876 0.16898554 0.21853095
  0.19194184 0.09672961 0.13800201 0.11169428 0.08497813 0.14310468
  0.09745476 0.1297168  0.10215642 0.12971029 0.1499749  0.14233879
  0.08141317 0.1569664  0.52409904 0.50305038 0.4131244  0.51267046]
 [0.06769193 0.16577987 0.13795876 0.10643924 0.13037731 0.16860306
  0.14808878 0.07462974 0.10647261 0.08617543 0.06556313 0.11040947
  0.07518921 0.10008033 0.07881669 0.10007531

In [158]:
print(np.average(mse))
print(np.average(anees))

2.354213918405176e-26
6.640725497043202e-09
